In [84]:
%reload_ext autoreload
%autoreload 2

import pandas as pd
pd.options.plotting.backend = "plotly"

import sys
sys.path.insert(0, 'driftpy/src/')

import driftpy
import numpy as np 
from sim.sim import SimpleDriftSim, load_hist_oracle

import os
import datetime
from sim.agents import * 
from programs.clearing_house.state import * 
from sim.events import OpenPositionEvent
from sim.helpers import random_walk_oracle, rand_heterosk_oracle, class_to_json
import pickle as cPickle

# initial setup 
- setup the agents in the simulation 
- setup the market/clearing house 

In [85]:
def make_clearing_house(
    base_spread, 
    oracle=None,
):
    # generate random oracle data 
    if oracle is None: 
        prices, timestamps = random_walk_oracle(1)
        oracle = Oracle(prices=prices, timestamps=timestamps)
    
    # create the market with/without spread 
    amm = AMM(
        oracle=oracle, 
        base_asset_reserve=int(10_000_000 * AMM_RESERVE_PRECISION), 
        quote_asset_reserve=int(10_000_000 * AMM_RESERVE_PRECISION),
        funding_period=60*60,
        peg_multiplier=int(oracle.get_price(0) * PEG_PRECISION),
        base_spread=base_spread
    )
    market = Market(amm)
        
    # create clearing house 
    fee_structure = FeeStructure(numerator=1, denominator=1000)
    clearing_house = ClearingHouse([market], fee_structure)
    clearing_house.name = f''

    return clearing_house
    
# agent which always pushes mark => oracle 
arb = Arb(
    intensity=0.6, 
    market_index=0, 
    user_index=0,
)
# random trader 
noise = Noise(
    intensity=1, 
    market_index=0, 
    user_index=0,
    size=1_000, # 1k trades
)

agents = [
    arb, 
    noise,
]
os.makedirs("sim-results/", exist_ok=True)


## simulation with no spread (base_spread=0)

In [86]:
# without spread sim 
no_spread_sim = SimpleDriftSim(
    "sim-results/no_spread",
    clearing_house=make_clearing_house(base_spread=0),
    agents=agents
)

no_spread_sim.run()
no_spread_result_df = no_spread_sim.to_df(save=True).add_prefix('noSpread_')
no_spread_result_df.head()

running simulation for 526 timesteps
running sim from timestamp 0 to 526
skipping time step: 0 ... ch time: 3
skipping time step: 1 ... ch time: 3
skipping time step: 2 ... ch time: 3
ERR: <class 'sumtypes.SHORT'> 11.971800000735485 11.971800000735485 11.9718000007 11.970676395798783
ERR: <class 'sumtypes.SHORT'> 11.97220000073528 11.97220000073528 11.9722000007 11.972000000000001
ERR: <class 'sumtypes.SHORT'> 11.97260000725313 11.97260000725313 11.9726000072 11.972
ERR: <class 'sumtypes.SHORT'> 11.972800013100102 11.972800013100102 11.9728000131 11.972000000000001
ERR: <class 'sumtypes.SHORT'> 11.97260000725313 11.97260000725313 11.9726000072 11.972
saving results to sim-results/no_spread...


100%|██████████| 1049/1049 [00:00<00:00, 4374.83it/s]


,noSpread_m0_mark_price,noSpread_m0_oracle_price,noSpread_m0_bid_price,noSpread_m0_ask_price,noSpread_m0_wouldbe_peg,noSpread_m0_wouldbe_peg_cost,noSpread_m0_predicted_long_funding,noSpread_m0_predicted_short_funding,noSpread_m0_last_mid_price_twap,noSpread_m0_repeg_to_oracle_cost,...,noSpread_u1_total_fee_rebate,noSpread_u1_open_orders,noSpread_u1_cumulative_deposits,noSpread_u1_total_collateral,noSpread_u0_m0_base_asset_amount,noSpread_u0_m0_quote_asset_amount,noSpread_u0_m0_last_cumulative_funding_rate,noSpread_u0_m0_last_funding_rate_ts,noSpread_u0_m0_upnl,noSpread_u0_m0_ufunding
0,11.9720,11.972001,11.9720,11.9720,11.972,0.0,-3.694979e-11,-3.694979e-11,11.972,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,11.9720,11.972001,11.9720,11.9720,11.972,0.0,-3.694979e-11,-3.694979e-11,11.972,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,11.9720,11.972001,11.9720,11.9720,11.972,0.0,-3.694979e-11,-3.694979e-11,11.972,0.0,...,0.0,0.0,1.000000e+13,1.000000e+13,NaN,NaN,NaN,NaN,NaN,NaN
3,11.9720,11.972001,11.9720,11.9720,11.972,0.0,-3.694979e-11,-3.694979e-11,11.972,0.0,...,0.0,0.0,1.000000e+13,1.000000e+13,NaN,NaN,NaN,NaN,NaN,NaN
4,11.9718,11.972001,11.9718,11.9718,11.972,0.0,-3.694979e-11,-3.694979e-11,11.972,0.0,...,0.0,0.0,1.000000e+13,1.000000e+13,-8.352893e+14,1.000000e+09,0.0,0.0,-0.995314,-0.0


In [87]:
no_spread_result_df[[
    'noSpread_m0_mark_price', 
    'noSpread_m0_oracle_price',
]].plot()

In [88]:
spread_values = pd.DataFrame(
    data={'spread': (
        no_spread_result_df[['noSpread_m0_ask_price']].values
        - no_spread_result_df[['noSpread_m0_bid_price']].values 
    ).flatten()}
)
spread_values.plot(title='base spread = 0: spread')

# simulation with a spread (base_spread=100)

In [89]:
# load the same oracle data from the no_spread sim 
oracle = Oracle("sim-results/no_spread/oracle_prices.csv")

# run again with spread sim 
spread_sim = SimpleDriftSim(
    "sim-results/spread",
    clearing_house=make_clearing_house(
        base_spread=100, # ! -- large spread 
        oracle=oracle
    ),
    agents=agents
)

spread_sim.run()
spread_result_df = spread_sim.to_df(save=True).add_prefix('withSpread_')
spread_result_df.head()

running simulation for 526 timesteps
running sim from timestamp 0 to 526
skipping time step: 0 ... ch time: 3
skipping time step: 1 ... ch time: 3
skipping time step: 2 ... ch time: 3
saving results to sim-results/spread...


100%|██████████| 1049/1049 [00:00<00:00, 4397.38it/s]


,withSpread_m0_mark_price,withSpread_m0_oracle_price,withSpread_m0_bid_price,withSpread_m0_ask_price,withSpread_m0_wouldbe_peg,withSpread_m0_wouldbe_peg_cost,withSpread_m0_predicted_long_funding,withSpread_m0_predicted_short_funding,withSpread_m0_last_mid_price_twap,withSpread_m0_repeg_to_oracle_cost,...,withSpread_u1_total_fee_rebate,withSpread_u1_open_orders,withSpread_u1_cumulative_deposits,withSpread_u1_total_collateral,withSpread_u0_m0_base_asset_amount,withSpread_u0_m0_quote_asset_amount,withSpread_u0_m0_last_cumulative_funding_rate,withSpread_u0_m0_last_funding_rate_ts,withSpread_u0_m0_upnl,withSpread_u0_m0_ufunding
0,11.9720,11.972001,11.971401,11.972599,11.972,0.0,-3.694979e-11,-3.694979e-11,11.972,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,11.9720,11.972001,11.971401,11.972599,11.972,0.0,-3.694979e-11,-3.694979e-11,11.972,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,11.9720,11.972001,11.971401,11.972599,11.972,0.0,-3.694979e-11,-3.694979e-11,11.972,0.0,...,0.0,0.0,1.000000e+13,1.000000e+13,NaN,NaN,NaN,NaN,NaN,NaN
3,11.9720,11.972001,11.971401,11.972599,11.972,0.0,-3.694979e-11,-3.694979e-11,11.972,0.0,...,0.0,0.0,1.000000e+13,1.000000e+13,NaN,NaN,NaN,NaN,NaN,NaN
4,11.9718,11.972001,11.971201,11.972399,11.972,0.0,-3.694979e-11,-3.694979e-11,11.972,0.0,...,0.0,0.0,1.000000e+13,1.000000e+13,-8.353311e+14,1.000000e+09,0.0,0.0,-50003.479304,-0.0


In [90]:
spread_df = pd.DataFrame(
    data={'spread': (
        spread_result_df[['withSpread_m0_ask_price']].values 
        - spread_result_df[['withSpread_m0_bid_price']].values 
    ).flatten()}
)
spread_df.plot(title='base spread = 100: spread')